In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *

Using TensorFlow backend.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

Stage 1 data: main

In [3]:
train_emb_api,train_emb , train_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/main_stage1_train.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/main_stage1_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/main_stage1_test.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

test of sent2vec vector: (431, 347, 768) (431, 347) (431, 347, 1)


In [4]:
# train only fam hash unique
def unique_hash(train_emb,train_emb_api,train_fam_ans,train_rep_ans):
    unique , indx = np.unique(train_emb, axis=0, return_index=True)
    emb_api = train_emb_api[indx]
    fam = train_fam_ans[indx]
    rep = train_rep_ans[indx]
    print(unique.shape,emb_api.shape,fam.shape)
    return unique,emb_api,fam,rep #改
# train_emb,train_emb_api,train_fam_ans,train_rep_ans = unique_hash(train_emb,train_emb_api,train_fam_ans,train_rep_ans)

In [5]:
def _shuffle(X, X2 ,X3):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize])

In [6]:
train_emb, train_emb_api, train_rep_ans = _shuffle(train_emb, train_emb_api, train_rep_ans)
# valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = _shuffle(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans)

# test_emb, test_emb_api,test_fam_ans,test_rep_ans  = _shuffle(test_emb,test_emb_api,test_fam_ans,test_rep_ans)

print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_rep_ans.shape)

# print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

train of sent2vec vector: (17437, 347, 768) (17437, 347) (17437, 347, 1)
valid of sent2vec vector: (437, 347, 768) (437, 347) (437, 347, 1)


Stage 2 data: pids

In [30]:
train_emb_api,train_emb , train_fam_ans = pickle.load(open('data/tree-rep-profiles-partial/pids_stage2_train.pkl','rb'))
valid_emb_api,valid_emb, valid_fam_ans = pickle.load(open('data/tree-rep-profiles-partial/pids_stage2_valid.pkl','rb'))
test_emb_api,test_emb ,test_fam_ans = pickle.load(open('data/tree-rep-profiles-partial/pids_stage2_test.pkl','rb'))

print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape)

train_emb, train_emb_api, train_rep_ans = _shuffle(train_emb, train_emb_api, train_fam_ans)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape)

test of sent2vec vector: (347, 347, 768) (347, 347) (347, 22)
train of sent2vec vector: (6251, 347, 768) (6251, 347) (6251, 22)
valid of sent2vec vector: (347, 347, 768) (347, 347) (347, 22)


In [31]:
sum(train_fam_ans)

array([ 176,   99,   90,  156,  133,  761,   88,  113,  304,  418,  389,
        197,  603,  243,  925,  102, 1614,  114,  102,  171,  232,  501])

In [10]:
# kkk = (emb_matrix - kk)/kk


In [11]:
# kk = np.std(emb_matrix,axis=-1)
# kk = np.expand_dims(kk,axis=-1)
# kk = np.repeat(kk,768,axis=-1)
# kk

In [12]:
# bc = SMOTEENN()
# N,t,d = train_emb.shape
# train_emb_ = train_emb.reshape(N,t*d)
# train_fam_ans_ = train_fam_ans.reshape(N,)
# train_emb_ , train_fam_ans_  = bc.fit_resample(train_emb_, train_fam_ans_)

In [13]:
# train_emb = train_emb_.reshape(-1,t,d)
# train_fam_ans = train_fam_ans_.reshape(-1,1)

In [34]:
class_weights = sum(train_fam_ans) / sum(sum(train_fam_ans))
fam_weights={}
for i in range(len(class_weights)):
    fam_weights[i] = 1/class_weights[i]
fam_weights

all_fam = sum(train_fam_ans)
for i in range(len(all_fam)):
    fam_weights[i] = all_fam[i]
fam_weights

import math
def create_class_weight(labels_dict,mu=0.34): #0.79 #0.5: only hash #改 #0.55: not only hash
    total = np.sum(np.array(list(labels_dict.values())))
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight
fam_weights = create_class_weight(fam_weights)
fam_weights

{0: 2.67748945769127,
 1: 3.252853602594832,
 2: 3.3481637823991566,
 3: 2.7981174454798845,
 4: 2.957624324507668,
 5: 1.2133400948677358,
 6: 3.370636638251215,
 7: 3.1205856340170812,
 8: 2.1309457513232,
 9: 1.8124920202046655,
 10: 1.8843941091109755,
 11: 2.564769723991433,
 12: 1.4460562560022363,
 13: 2.3549120093888734,
 14: 1.0181797152169965,
 15: 3.2230006394451505,
 16: 1.0,
 17: 3.1117750043349264,
 18: 3.2230006394451505,
 19: 2.7063098962267618,
 20: 2.4012360810631117,
 21: 1.6313673516445568}

In [15]:
fam_weight = {}
fam_weight['family'] = fam_weights

## model hyperparameters

In [90]:
emb_dim = 768 #被除數
num_heads = 48#除數，要整除
max_length = 347 # max sequence length
fam_num = train_fam_ans.shape[1]
vocabulary_size = 26
transformer_depth = 1
transformer_dropout = 0.1
l2_reg_penalty = 1e-6#1e-4
dp_rate = 0.01

traina = True #改
batch_size = 128 #改

In [91]:
constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
init = keras.initializers.Ones()
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding')
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

# Model

In [92]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
#shape=(max_length,emb_dim),,batch_shape=(batch_size,max_length,emb_dim)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
#shape=(max_length,),batch_shape=(batch_size,max_length)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='api_emb')(sent_ids) #改



In [93]:
segment_embeddings = Add()([sentemb,api_emb])
# next_step_input1 = RemoveMask()(segment_embeddings)
# next_step_input = coordinate_embedding_layer(next_step_input1, step=0,trainable=traina) #next_step_input_emb
# next_step_input= RestoreMask()([next_step_input,segment_embeddings])
# next_step_input = add_segment_layer([next_step_input, api_emb]) 

att_rnn,state_h_g = GRU(int(emb_dim/4),return_sequences=True,return_state=True,name='common_extract'
                      ,trainable=True)(segment_embeddings)
next_step_input = BatchNormalization(name='bn_common')(att_rnn)
att_in = TimeDistributed(Dense(64,kernel_initializer=keras.initializers.lecun_normal(),activation='selu', # 改
               name='attention_mlp',trainable=False),name='in_rep')(next_step_input)
rep_prediction = (
        TimeDistributed(Dense(1, name='0_1_predict', activation='sigmoid',trainable=False),name='out_rep') # hard_sigmoid
    (att_in))

# state = Concatenate()([state_h,state_c])

# bn = BatchNormalization(name='bn_state')(state)
# state = Dense(int(emb_dim/32),kernel_initializer=keras.initializers.lecun_normal(),activation='selu')(bn)


In [94]:


mul = multiply([att_rnn,rep_prediction],name='mul')
mul = BatchNormalization()(mul)
lstm,state_h = GRU(int(emb_dim/8),return_sequences=False,return_state=True,name='family_in')(mul)
# att_layer = MultiHeadAttention(
#     head_num=4, trainable=True,
#     name='Multi-Head')(mul)
lstm = BatchNormalization()(lstm)
alls=[]
for i in range(fam_num):
    alls.append(Dense(1,activation='sigmoid') 
    (Dense(16,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
    (BatchNormalization()
    (Dense(64,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')(lstm)
    
    ))))



# lstm,state_h = GRU(int(emb_dim/8),return_sequences=False,return_state=True,name='family_in')(mul)

# # state = Concatenate()([state_h,state_c])
# state = Dense(4,activation='selu',kernel_initializer='lecun_normal')(state_h)
# state = BatchNormalization()(state)
# alls=[]
# for i in range(fam_num):
#     alls.append(Dense(1,activation='sigmoid') 
#                 (Dense(32,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
#                  ( (Concatenate()([BatchNormalization()(Dropout(dp_rate)
#                                                        (Dense(64
#                                                               ,kernel_initializer=keras.initializers.lecun_normal()
#                                                               ,activation='selu')(BatchNormalization()(lstm))
#                                                        )),state])))))
#                      (Dropout(dp_rate)
#                 (Dense(128,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
#                  (BatchNormalization()
#                   (lstm)
#                   ))))))
out = Concatenate(name='family')(alls)


'''att_layer = BatchNormalization(name='bn1')(att_layer)
head_out = MultiHead(Dense(4,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
                     ,layer_num=2,name='Multi-Dense')(att_layer)
head_out1 = RemoveMask()(head_out)
flat = Flatten()(head_out1)
# flat = RestoreMask()([flat,att_layer])
con = Concatenate()([state_h,state_c])
flat = Concatenate()([con,flat])
bn = BatchNormalization(name='bn2')(flat)
dense2 = Dense(1024,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
              name='dense2')(bn)
dp = Dropout(dp_rate)(dense2)
out = Dense(44,activation='sigmoid',name='family_out')(dp)'''
                     
# bn = BatchNormalization()
# dp = Dropout(dp_rate)
# dense1 = Dense(int(emb_dim/8),kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
#               kernel_regularizer=l2_regularizer,name='dense1')
# dense2 = Dense(32,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
#               name='dense2')
# dense3 = Dense(1,kernel_initializer=keras.initializers.lecun_normal(),activation='sigmoid',
#               name='dense3')
# gru = GRU(int(emb_dim/4), dropout=dp_rate, recurrent_dropout=dp_rate,name='gru_64')
# alls = []
# for i in range(fam_num):
#     alls.append(dense2(bn(gru(mul))))
# #     alls.append(dense3(dense2(dp(bn(dense1(BatchNormalization()(gru(mul))))))))
# out = Concatenate()(alls)
# out = BatchNormalization()(out)
# bn = BatchNormalization()
# all_out = []
# for i in range(fam_num):
#     all_out.append(dense3(bn(dense1(out))))
# out1 = Concatenate()(all_out)
# # out = Dense(44,activation='sigmoid',name='family_out')(out)

"att_layer = BatchNormalization(name='bn1')(att_layer)\nhead_out = MultiHead(Dense(4,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')\n                     ,layer_num=2,name='Multi-Dense')(att_layer)\nhead_out1 = RemoveMask()(head_out)\nflat = Flatten()(head_out1)\n# flat = RestoreMask()([flat,att_layer])\ncon = Concatenate()([state_h,state_c])\nflat = Concatenate()([con,flat])\nbn = BatchNormalization(name='bn2')(flat)\ndense2 = Dense(1024,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',\n              name='dense2')(bn)\ndp = Dropout(dp_rate)(dense2)\nout = Dense(44,activation='sigmoid',name='family_out')(dp)"

In [95]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[out]) #out

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 347)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 347, 768)     0                                            
__________________________________________________________________________________________________
masking_12 (Masking)            (None, 347)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_11 (Masking)            (None, 347, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [96]:
model.load_weights('./model/stages_gru_att/1stStage_22fam_0620_gru_sent2vec_main.h5',by_name=True) #改
# model = multi_gpu_model(model , gpus=2)

# model.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')

In [23]:
'''

sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)

final_emb = Add()([sentemb,api_emb])

timesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb
state = Concatenate()([state_h,state_c])
fc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)
fc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)
fc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)
# fc = keras.backend.repeat_elements(fc,256,axis=-1)
# fc = keras.backend.expand_dims(fc,axis=-1)
mul = Multiply()([fc,timesteps])
# mul = BatchNormalization()(mul)
alls = []
gru = (GRU(int(emb_dim/4))) #/8
# gru = GRU(1)
bn = BatchNormalization()
dp = Dropout(0.01)

dense = Dense(1,activation='sigmoid')
for i in range(fam_num):
#     alls.append(dense(bn(gru(mul))))
    alls.append(dense(dp(bn(gru(mul)))))
#     alls.append(gru(mul))
out = Concatenate(name='family')(alls)
# out = Dense(44,activation='sigmoid')(out)
model_old = Model(inputs=[sent_ids1,sentemb1], outputs=[out]) #out
model_old = multi_gpu_model(model_old , gpus=3)
model_old.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')
model_old.summary()

# model = load_model('./model/LSTM_att/1stStage_44fam_0607.h5_all.h5')'''

"\n\nsentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')\nsentemb = Masking(mask_value=0)(sentemb1)\nsent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID\nsent_ids = Masking(mask_value=0)(sent_ids1)\napi_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)\n\nfinal_emb = Add()([sentemb,api_emb])\n\ntimesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb\nstate = Concatenate()([state_h,state_c])\nfc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)\nfc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)\nfc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)\n# fc = keras.backend.repeat_elements(fc,256,axis=-1)\n# fc = keras.backend.expand_dims(fc,axis=-1)\nmul = Multiply()([fc,timest

In [24]:
# model_old.layers[-2].save_weights('./model/LSTM_att/test.h5')

In [97]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy
def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score

# 訓練參數
los = [hn_multilabel_loss,binary_focal_loss(alpha=.25, gamma=2)] # 1st stage.  f1_loss
#SINGLE
los = [binary_focal_loss(alpha=.25, gamma=2)] #改
# los = [hn_multilabel_loss]
# los = [losses.binary_crossentropy]
# MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'out_rep': bin_acc,'family': f1_metric} # 1st stage. km.f1_score()
#SINGLE
metric = [f1_metric,bin_acc]
# metric = [km.f1_score(),bin_acc,km.binary_f1_score()]
# metric = {'RasMMA': 'acc'}
# metric = [bin_acc] #改
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [1,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 0.001#2e-4 # 2nd stage: 1e-4 @1st:2e-4 0.002
# batch_size = 128 #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/stages_gru_att/2ndStage_22fam_0620_gru_sent2vec_main.h5' #改
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

In [99]:
# import tensorflow as tf
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False) #clipnorm=1. , clipvalue=1.
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #改
# tf.keras.optimizers.Nadam
lr_scheduler1 = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=0.001, lr_low=1e-8, #learning_rate #改
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min')

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [100]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path, #val_f1_metric,max。val_family_f1_metric
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True), #改
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler,history,stop_nan ,lr_scheduler1
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

In [101]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1):#, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
#             train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield(train_X2,train_Y2)
#             yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
            yield ([train_X1,train_X2],[train_Y1]) #改
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [102]:

H = model.fit_generator( #train_rep_ans
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_fam_ans)#,Y_train2=train_fam_ans) #Y_train2
#                     generator=training_batch(batch_size=batch_size,X_train1=valid_emb_api,X_train2=valid_emb ,
#                                              Y_train1=train_rep_ans,Y_train2=train_fam_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_fam_ans]) #ori
#                    ,validation_data= (valid_emb, valid_fam_ans) 
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) #ori #改
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10 ,class_weight=fam_weights#[None,fam_weights] #改
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

/home/leoqaz12/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
49/49 [==============================] - 72s 1s/step - loss: 253.8209 - f1_metric: 0.0836 - bin_acc: 0.8818 - val_loss: 60.9012 - val_f1_metric: 0.0089 - val_bin_acc: 0.9434

Epoch 00001: val_loss improved from inf to 60.90122, saving model to ./model/stages_gru_att/2ndStage_22fam_0620_gru_sent2vec_main.h5
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009999975326256032.
49/49 [==============================] - 55s 1s/step - loss: 83.1387 - f1_metric: 0.0362 - bin_acc: 0.9452 - val_loss: 58.1083 - val_f1_metric: 0.0089 - val_bin_acc: 0.9424

Epoch 00002: val_loss improved from 60.90122 to 58.10833, saving model to ./model/stages_gru_att/2ndStage_22fam_0620_gru_sent2vec_main.h5
Epoch 3/1000

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009999901305267642.
49/49 [==============================] - 55s 1s/step - loss: 77.5100 - f1_metric: 0.0899 - bin_acc: 0.946

49/49 [==============================] - 54s 1s/step - loss: 76.9870 - f1_metric: 0.0974 - bin_acc: 0.9482 - val_loss: 57.2702 - val_f1_metric: 0.0092 - val_bin_acc: 0.9443

Epoch 00025: val_loss did not improve from 55.20895
Epoch 26/1000

Epoch 00026: LearningRateScheduler setting learning rate to 0.0009984586822798954.
49/49 [==============================] - 54s 1s/step - loss: 76.4096 - f1_metric: 0.1037 - bin_acc: 0.9481 - val_loss: 63.8192 - val_f1_metric: 0.0000e+00 - val_bin_acc: 0.9454

Epoch 00026: val_loss did not improve from 55.20895
Epoch 27/1000

Epoch 00027: LearningRateScheduler setting learning rate to 0.0009983329806873748.
49/49 [==============================] - 54s 1s/step - loss: 79.2311 - f1_metric: 0.0674 - bin_acc: 0.9477 - val_loss: 62.7621 - val_f1_metric: 0.0093 - val_bin_acc: 0.9439

Epoch 00027: val_loss did not improve from 55.20895
Epoch 28/1000

Epoch 00028: LearningRateScheduler setting learning rate to 0.000998202360798868.
49/49 [==================

49/49 [==============================] - 54s 1s/step - loss: 74.3469 - f1_metric: 0.0540 - bin_acc: 0.9468 - val_loss: 59.3764 - val_f1_metric: 0.0046 - val_bin_acc: 0.9437

Epoch 00050: val_loss did not improve from 53.50711
Epoch 51/1000

Epoch 00051: LearningRateScheduler setting learning rate to 0.000993844231855866.
49/49 [==============================] - 54s 1s/step - loss: 76.1332 - f1_metric: 0.0763 - bin_acc: 0.9468 - val_loss: 60.3807 - val_f1_metric: 0.0000e+00 - val_bin_acc: 0.9450

Epoch 00051: val_loss did not improve from 53.50711
Epoch 52/1000

Epoch 00052: LearningRateScheduler setting learning rate to 0.0009935960710373397.
49/49 [==============================] - 54s 1s/step - loss: 80.9704 - f1_metric: 0.0826 - bin_acc: 0.9480 - val_loss: 57.9641 - val_f1_metric: 0.0094 - val_bin_acc: 0.9441

Epoch 00052: val_loss did not improve from 53.50711
Epoch 53/1000

Epoch 00053: LearningRateScheduler setting learning rate to 0.000993343038674213.
49/49 [===================

49/49 [==============================] - 54s 1s/step - loss: 78.0307 - f1_metric: 0.0675 - bin_acc: 0.9466 - val_loss: 56.6347 - val_f1_metric: 0.0000e+00 - val_bin_acc: 0.9452

Epoch 00075: val_loss did not improve from 53.50711
Epoch 76/1000

Epoch 00076: LearningRateScheduler setting learning rate to 0.0009861850983492364.
49/49 [==============================] - 54s 1s/step - loss: 79.8076 - f1_metric: 0.0472 - bin_acc: 0.9464 - val_loss: 58.3516 - val_f1_metric: 0.0046 - val_bin_acc: 0.9441

Epoch 00076: val_loss did not improve from 53.50711
Epoch 77/1000

Epoch 00077: LearningRateScheduler setting learning rate to 0.0009858160082986723.
49/49 [==============================] - 54s 1s/step - loss: 77.2482 - f1_metric: 0.0452 - bin_acc: 0.9471 - val_loss: 59.4859 - val_f1_metric: 0.0000e+00 - val_bin_acc: 0.9446

Epoch 00077: val_loss did not improve from 53.50711
Epoch 78/1000

Epoch 00078: LearningRateScheduler setting learning rate to 0.0009854421234895864.
49/49 [=============

# Evaluate
* multi model weights 儲存

In [30]:
# model_ = model.layers[-2]
# model_.load_weights(model_save_path)
# model_.summary()

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 347)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 347, 768)     0                                            
__________________________________________________________________________________________________
masking_4 (Masking)             (None, 347)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_3 (Masking)             (None, 347, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [25]:
# model.load_weights(model_save_path)
# model_ = model.layers[-3]
# model_.save_weights(model_save_path)
# model_.summary()
pickle.dump(file=open(tensorboard_log_path + '/'+'1stStage_H.pkl','wb'),obj=H) #改


NameError: name 'H' is not defined

In [24]:
from keras.models import model_from_json, model_from_yaml
json_string = model.to_json()
yaml_string = model.to_yaml()
pickle.dump(file=open(tensorboard_log_path + '/'+'1stStage_arch.pkl','wb'),obj=(json_string,yaml_string))

In [59]:
# test_emb_,test_emb_api_,test_fam_ans_,test_rep_ans_ = unique_hash(test_emb,test_emb_api,test_fam_ans,test_rep_ans)

(292, 213, 768) (292, 213) (292, 44)


In [61]:
# model.load_weights('./model/LSTM_att/1stStage_44fam_0610.h5')
# model.load_weights('./model/att_clf/1stStage_44fam_0611_copy1.h5')
# score = model.evaluate([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2)
print(len(test_emb_api)) #改
ans = model.predict([test_emb_api,test_emb]) #改
y_true = test_fam_ans #改
# ans = model.predict([valid_emb_api,valid_emb])
len(ans)

347


347

In [28]:
test_rep_ans[113]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [62]:
ans[113]

array([0.12224409, 0.1362347 , 0.17499596, 0.09577695, 0.1950016 ,
       0.2373001 , 0.10070118, 0.07382572, 0.14515352, 0.22195691,
       0.24388993, 0.09415463, 0.22798213, 0.14712897, 0.23245889,
       0.12479731, 0.27729812, 0.16044661, 0.09510088, 0.13270906,
       0.18814352, 0.20053473], dtype=float32)

In [37]:
ans[0][14]

array([[0.6530128 ],
       [0.6269398 ],
       [0.504724  ],
       [0.5047246 ],
       [0.504725  ],
       [0.5047249 ],
       [0.38633832],
       [0.5844016 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.5844019 ],
       [0.9855267 ],
       [0.584

In [65]:
model.load_weights(model_save_path)

In [ ]:
# model_.save_weights('./model/LSTM_att/test4.h5')

In [74]:
# model_ = model.layers[-2]
layer_name = 'lambda_1' #lambda_1 multiply_1  #9~12
intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.layers[12].output)
intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
intermediate_output[113]

IndexError: list index out of range

In [77]:
intermediate_output[0][0]

array([-0.        ,  0.        ,  0.        , -1.        ,  0.        ,
        0.        , -1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.7615941 , -0.        ,  1.        ,  0.7615942 ,
        1.        , -0.76159376,  1.        ,  0.        ,  0.        ,
        1.        , -0.        ,  0.        ,  0.        ,  1.        ,
        0.76119554, -0.        , -1.        , -1.        , -1.        ,
       -1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        1.        , -1.        ,  0.        , -0.        ,  1.        ,
        1.        , -1.        ,  0.        ,  1.        , -0.7615942 ,
        0.        ,  0.        ,  0.        , -1.        ,  0.        ,
        0.        , -0.        ,  0.7615942 , -0.7615942 , -0.7615942 ,
        0.7615916 ,  1.        , -0.7615942 , -0.7615942 ,  1.        ,
        0.7615942 ,  0.        , -1.        ,  0.        ,  0.        ,
        0.        , -0.7615942 , -0.7615919 ,  0.        ,  0.76

In [103]:
train_emb_api.shape

(6251, 347)

In [104]:
kk = train_emb_api.reshape(1,2169097)
set(kk[0])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}

In [ ]:
intermediate_output[113].shape

In [ ]:
intermediate_output[190]

In [ ]:
model_.summary() #multiply_1 

In [ ]:
sum(intermediate_output)

In [77]:
from sklearn.metrics import f1_score,recall_score,precision_score,hamming_loss
# y_true = np.squeeze(test_fam_ans)
# y_true = np.squeeze(valid_fam_ans1)
# y_pred = np.squeeze(predict_fam)
final_ans = []
for sample in ans:
    sample_ans = []
    for value in sample:
        if value < 0.3: #0.26 #0.33
            sample_ans.append(0)
        else:
            sample_ans.append(1)
    final_ans.append(sample_ans)
final_ans = np.array(final_ans)
print(final_ans.shape , sum(final_ans[0]))

(347, 22) 1


In [69]:
from sklearn.metrics import roc_auc_score , roc_curve, auc , precision_recall_curve
fam_id = 0
area_score = roc_auc_score(y_true,ans,average='micro')
fp_rate , tp_rate , thresholds = roc_curve(y_true[:,fam_id],ans[:,fam_id])
auc_score = auc(fp_rate,tp_rate)
area_score , auc_score

(0.7222781080068793, 0.47810945273631844)

In [80]:
for i,v in zip(sum(y_true),sum(final_ans)):
    print(i,v)

12 0
4 0
5 0
7 1
9 0
41 27
5 1
6 2
16 7
25 2
25 6
7 3
31 7
13 1
48 25
7 0
94 185
4 2
3 1
7 1
11 1
33 4


In [79]:
sum(final_ans)

array([  0,   0,   0,   1,   0,  27,   1,   2,   7,   2,   6,   3,   7,
         1,  25,   0, 185,   2,   1,   1,   1,   4])

In [78]:
# y_true = test_fam_ans
print(y_true.shape , final_ans.shape)
recall = recall_score(y_true=y_true, y_pred=final_ans, average='micro')
precision = precision_score(y_true=y_true, y_pred=final_ans, average='micro')
f1 = f1_score(y_true=y_true, y_pred=final_ans, average='micro')
h_loss = hamming_loss(y_true,final_ans)
recall ,precision, f1 , h_loss

(347, 22) (347, 22)


(0.15254237288135594,
 0.22826086956521738,
 0.18287373004354138,
 0.07374901755305213)